# MCMC5.5: Exact Digonalization in Python

This is FYI. I am currently using Python 3.7.1. See also https://github.com/jhauschild/lecture_comp_methods/blob/master/2_exact_diagonalization/sol4_simple_ED.ipynb

In [1]:
import numpy as np
from scipy import sparse
from scipy.sparse.linalg import eigsh
from itertools import chain
Lx = 2
Ly = 2
Jx = 1 / 3
Jy = 1 / 3
Jz = 1 / 3
def spinoperators(sigma, i, N):
    if i == 0 and N == 1:
        return sigma
    elif N == 1:
        return sparse.csr_matrix(np.eye(2), dtype=complex)
    elif i == N - 1:
        return sparse.kron(spinoperators(sigma, i, N - 1), sigma, 'csr')
    else:
        return sparse.kron(spinoperators(sigma, i, N - 1), np.eye(2), 'csr')

sigma_x = sparse.csr_matrix(np.array([[0., 1.], [1., 0.]]), dtype=complex)
sigma_y = sparse.csr_matrix(np.array([[0., -1.j], [1.j, 0.]]), dtype=complex)
sigma_z = sparse.csr_matrix(np.array([[1., 0.], [0., -1.]]), dtype=complex)
N = 2 * Lx * Ly
nnx = zip(range(0, N - 1, 2), range(1, N, 2))
nny = chain.from_iterable(zip(range(2 * i + 2, 2 * Lx * Ly + 2 * i + 2, 2 * Lx), range(2 * i +1, 2 * Lx * Ly  + 2 * i  + 1, 2 * Lx)) for i in range(Lx - 1))
nnz = zip(range(0, N - 1, 2), chain(range(2 * Lx + 1, N, 2), range(1, 2 * Lx, 2)))
sigma_x_list = [spinoperators(sigma_x, i, N) for i in range(N)]
sigma_y_list = [spinoperators(sigma_y, i, N) for i in range(N)]
sigma_z_list = [spinoperators(sigma_z, i, N) for i in range(N)]
H = sparse.csr_matrix((1 << N, 1 << N), dtype=complex)
for (J, nn, sigma_list) in zip([Jx, Jy, Jz], [nnx, nny, nnz], [sigma_x_list, sigma_y_list, sigma_z_list]):
    for bond in nn:
        H += J * sigma_list[bond[0]] * sigma_list[bond[1]]
H

<256x256 sparse matrix of type '<class 'numpy.complex128'>'
	with 1696 stored elements in Compressed Sparse Row format>

In [2]:
e, v = eigsh(H, k = 200, which = 'SA', return_eigenvectors = True, tol=1e-4)
e

array([-2.00000000e+00, -2.00000000e+00, -2.00000000e+00, -2.00000000e+00,
       -1.70770188e+00, -1.70770188e+00, -1.70770188e+00, -1.70770188e+00,
       -1.04103521e+00, -1.04103521e+00, -1.04103521e+00, -1.04103521e+00,
       -1.49071198e+00, -1.49071198e+00, -1.49071198e+00, -1.49071198e+00,
       -1.49071198e+00, -1.49071198e+00, -1.49071198e+00, -1.49071198e+00,
       -1.76323270e+00, -1.76323270e+00, -1.76323270e+00, -1.76323270e+00,
       -1.33333333e+00, -1.33333333e+00, -1.33333333e+00, -1.33333333e+00,
       -1.33333333e+00, -1.33333333e+00, -1.33333333e+00, -1.76323270e+00,
       -1.76323270e+00, -1.76323270e+00, -1.33333333e+00, -1.76323270e+00,
       -1.76323270e+00, -1.76323270e+00,  6.66666667e-01,  6.66666667e-01,
       -1.76323270e+00, -1.76323270e+00, -1.76323270e+00,  6.66666667e-01,
       -1.76323270e+00, -1.76323270e+00,  6.66666667e-01,  9.63664153e-01,
        9.63664153e-01, -1.76323270e+00,  6.66666667e-01,  6.66666667e-01,
        6.66666667e-01,  